In [1]:
# импортируем нужные библиотеки 

from sqlalchemy import create_engine
import pandas as pd
from locale import normalize

In [5]:
# функция берет данные из SQL

def sql_pull(sql, cred):
    ''' 
        Sql data grabber from samo / anaplan server 
    '''

    if cred == 'Anaplan':
        server = 'pvm-bi-sql-01'
        database = 'TUIOlap'

    else:
        server = 'pck-sam-sql-04'
        database = 'TUI'

    driver = 'ODBC Driver 17 for SQL Server'
    
    db_con = f'mssql://{server}/{database}?driver={driver}'
    engine = create_engine(db_con)
    con = engine.connect()

    data = pd.read_sql_query(sql, con)

    con.close()
    engine.dispose()

    return data

In [6]:
# Забираем данные из САМО SQL, предварительно нашли номера ревизий за 2 дня по схожим параметрам (рынок КЗ, один автор, направление Таиланд)

thai_kz =   '''
            SELECT 
                revision 'rev_number',
                tour 'package',
                townfrom 'departure',
                state 'state',
                datebeg 'check_in',
                nights 'duration',
                rev_value_adl 'rev_adl',
                rev_value_chd 'rev_chd',
                CAST(activedate AS DATE) 'activation'

            FROM
                    rev_detail

            WHERE
                    revision IN (1271763,1271764,1271765,1271766,1271767,1272772,1272773,1273110,1273249,1273250,1274686,
                    1274687,1274749,1274761,1276173,1276397,1276535,1276548,1276577,1276595,1276645,1276660)           
            '''

In [7]:
# Забираем данные
thai_kz = sql_pull(thai_kz, 'Samo')

# data cleaning
thai_kz.activation = pd.to_datetime(thai_kz.activation)

# создаем 2 датафрейма отфильровав по датам, будем сравнивать ревизии за 2 дня
first = thai_kz[thai_kz.activation == '2022-08-11']
second = thai_kz[thai_kz.activation == '2022-08-12']

# мерджим 2 датафрейма по основным параметрам (пакет, город отправления, страна, дата заезда, продолжительность, гостиница в ревизиях указанна = ВСЕ)
merged_kz = pd.merge(left = first, right = second, how = 'inner', suffixes = ('_first','_second'), on = ['package', 'departure', 'state', 'check_in', 'duration'])

# смотрим на шаг ревизии (были ли изменения между 2 днями)
merged_kz['rev_step_adl'] = merged_kz.rev_adl_second - merged_kz.rev_adl_first

# убираем ненужные колонки
merged_kz = merged_kz[['rev_number_first', 'rev_number_second', 'rev_chd_second', 'package', 'departure', 'state', 'check_in',
       'duration', 'rev_adl_first', 'rev_adl_second', 'rev_step_adl']]

# создаем колонку для удобства фильтрации по признаку шага ревизии (0 - не было изменений в ревизии, 1 - были изменения в ревизи)
merged_kz['false_change'] = 0
merged_kz.loc[merged_kz.rev_step_adl == 0, 'false_change'] = 1

# результат в %
merged_kz.false_change.value_counts(normalize = True)

1    0.767025
0    0.232975
Name: false_change, dtype: float64